In [1]:
# MODEL_CKPT_DIR = 'model_params_big_test'
# MODEL_CKPT_DIR = 'res_balanced_accuracy'
# MODEL_CKPT_DIR = 'res_f1'

# MODEL_CKPT_DIR = 'stratified_clusters=2'
# MODEL_CKPT_DIR = 'sims=True_stratified_clusters=2'

# MODEL_CKPT_DIR = 'stratified_clusters=9'
MODEL_CKPT_DIR = 'sims=True_stratified_clusters=9'

# LOAD_FIT_ON_DEV = False
LOAD_FIT_ON_DEV = True

# FILTER_POSITIVE_LABEL = True
FILTER_POSITIVE_LABEL = False

FILTER_CATEGORY_ID = 5
# FILTER_CATEGORY_ID = None

In [2]:
import os
import pandas as pd
import joblib
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score
from pathlib import Path


if LOAD_FIT_ON_DEV:
    xgb_model_path = Path(MODEL_CKPT_DIR) / 'xgboost_model_fit_on_dev.json'
    scaler_path = Path(MODEL_CKPT_DIR) / 'std_scaler_fit_on_dev.bin'
    
else:
    xgb_model_path = Path(MODEL_CKPT_DIR) / 'xgboost_model.json'
    scaler_path = Path(MODEL_CKPT_DIR) / 'std_scaler.bin'

# Load the scaler and the pre-trained XGBoost model.
scaler = joblib.load(scaler_path)
model = xgb.XGBClassifier()
model.load_model(xgb_model_path)

In [3]:
# Load the model data
MODEL_DATA_FILE = Path(MODEL_CKPT_DIR) / 'data.csv'
TEST_MODEL_DATA_FILE = Path(MODEL_CKPT_DIR) / 'data_test.csv'

enforced_dtypes = {'iseq_brand': 'int64'}

df_model = pd.read_csv(MODEL_DATA_FILE).astype(enforced_dtypes)
df_model_test = pd.read_csv(TEST_MODEL_DATA_FILE).astype(enforced_dtypes)

In [4]:
df_model_test_results = pd.read_csv(Path(MODEL_CKPT_DIR) / 'test_results.csv')
df_model_test_results


,accuracy,f1_score,precision,recall
0,0.849246,0.822701,0.844344,0.809516


In [5]:
df_model_test_results_per_category = pd.read_csv(Path(MODEL_CKPT_DIR) / 'test_results_per-category.csv')
df_model_test_results_per_category.set_index('category_id', inplace=True)
df_model_test_results_per_category

,category_name,category_size,accuracy,f1_score,precision,recall
category_id,,,,,,
0,"Одежда женская (платья, юбки, блузки, кофты, б...",404,0.861386,0.845919,0.866523,0.834594
1,"Одежда мужская (рубашки, футболки, брюки, шорт...",12,0.583333,0.555556,0.562500,0.557143
2,"Одежда гимнастическая (гимнастическая форма, о...",56,0.892857,0.849732,0.849732,0.849732
5,"Карта и путеводители (карты настенные, карты с...",65,0.861538,0.695471,0.706140,0.686508
6,"Товары для уборки (перчатки резиновые, швабры,...",429,0.834499,0.807758,0.829545,0.795314
7,"Товары для готовки (соевые соусы, ...)",3,0.666667,0.666667,0.750000,0.750000
8,"Мебель (столы, стулья, диваны, кровати, ...)",11,1.000000,1.000000,1.000000,1.000000
9,Аксессуары для компьютеров (флешки с гравировк...,3,0.666667,0.666667,0.750000,0.750000
10,"Всё остальное (игра настольная для детей, игру...",12,0.916667,0.899160,0.944444,0.875000


In [6]:
df_model_dev_results = pd.read_csv(Path(MODEL_CKPT_DIR) / 'dev_results.csv')
df_model_dev_results

,accuracy,f1_score,precision,recall
0,0.787126,0.728259,0.79533,0.714463


In [7]:
SPLIT_IDX = 0

df_model_dev_results_split = pd.read_csv(Path(MODEL_CKPT_DIR) / f'dev_results_split={SPLIT_IDX}.csv')
df_model_dev_results_split

,accuracy,f1_score,precision,recall
0,0.812359,0.779539,0.798396,0.768456


In [8]:
df_model_dev_results_per_category = pd.read_csv(Path(MODEL_CKPT_DIR) / 'dev_results_per-category.csv')
df_model_dev_results_per_category.set_index('category_id', inplace=True)
df_model_dev_results_per_category

,category_name,category_size,accuracy,f1_score,precision,recall
category_id,,,,,,
0,"Одежда женская (платья, юбки, блузки, кофты, б...",538,0.763474,0.707188,0.790431,0.701660
1,"Одежда мужская (рубашки, футболки, брюки, шорт...",16,0.594737,0.478839,0.586844,0.551515
2,"Одежда гимнастическая (гимнастическая форма, о...",75,0.864826,0.774905,0.864752,0.738490
5,"Карта и путеводители (карты настенные, карты с...",87,0.842517,0.604218,0.739230,0.620002
6,"Товары для уборки (перчатки резиновые, швабры,...",572,0.793494,0.746118,0.797231,0.734784
7,"Товары для готовки (соевые соусы, ...)",3,0.166667,0.161905,0.233333,0.208333
8,"Мебель (столы, стулья, диваны, кровати, ...)",15,1.000000,1.000000,1.000000,1.000000
9,Аксессуары для компьютеров (флешки с гравировк...,3,0.277778,0.245238,0.263889,0.400000
10,"Всё остальное (игра настольная для детей, игру...",15,0.885234,0.850883,0.917216,0.837179


In [9]:
SPLIT_IDX = 2

df_model_dev_results_per_category_split = pd.read_csv(Path(MODEL_CKPT_DIR) / f'dev_results_per_category_split={SPLIT_IDX}.csv')
df_model_dev_results_per_category_split.set_index('category_id', inplace=True)
df_model_dev_results_per_category_split

,category_name,category_size,accuracy,f1_score,precision,recall
category_id,,,,,,
0,"Одежда женская (платья, юбки, блузки, кофты, б...",548,0.728102,0.641788,0.763088,0.641827
1,"Одежда мужская (рубашки, футболки, брюки, шорт...",14,0.500000,0.426901,0.730769,0.562500
2,"Одежда гимнастическая (гимнастическая форма, о...",78,0.871795,0.811958,0.892308,0.776942
5,"Карта и путеводители (карты настенные, карты с...",92,0.826087,0.507363,0.912088,0.529412
6,"Товары для уборки (перчатки резиновые, швабры,...",567,0.813051,0.778842,0.803496,0.765896
7,"Товары для готовки (соевые соусы, ...)",3,0.000000,0.000000,0.000000,0.000000
8,"Мебель (столы, стулья, диваны, кровати, ...)",9,1.000000,1.000000,1.000000,1.000000
9,Аксессуары для компьютеров (флешки с гравировк...,3,0.333333,0.250000,0.166667,0.500000
10,"Всё остальное (игра настольная для детей, игру...",12,0.833333,0.777778,0.900000,0.750000


In [43]:
# > AVERAGE FILES HERE
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500.csv'
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_all_query_pairs.csv'
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-5_query_pairs.csv'

# > GOOD FILES HERE
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-5_query-23_nonquery-5539_pairs.csv'
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-5_query-23_nonquery-5539_embedded.csv'
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-20_query-23_nonquery-5539_embedded.csv'
DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-50_query-23_nonquery-5539_embedded.csv'
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-50_query-23_nonquery-5539_pairs.csv'

# > FILES W/INNER RUCLIP AS RANKER
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-30_query-2_nonquery-6_embedded.csv'
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-20query-23_nonquery-5539_embedded_sbert=all-distilroberta-v1_clip=siamese_contrastive.pt.csv'
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-20_query-2_nonquery-6_embedded_sbert=all-distilroberta-v1_clip=siamese_contrastive.pt_final-embs.csv'
DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-50_query-23_nonquery-5539_embedded_sbert=all-distilroberta-v1_clip=siamese_contrastive.pt_final-embs.csv'

DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-None_query=23_nonquery=5539_embedded_sbert=ru-en-RoSBERTa_clip=ruclip-vit-base-patch32-384.csv'

############################################################################

df_custom = pd.read_csv(DATA_FILE).astype(enforced_dtypes)

# Run inference

# Samples

In [44]:
import shap
import numpy as np
import matplotlib.pyplot as plt

def print_samples(
    matches_df,
    mode='predicted_matches_head',
    top_k=5,
    display_stats=False,
    display_name=False,
    display_shap=False,
    agg_waterfall=True,
    waterfalls=False,
    shap_values=False,
    stats_fmt="table"
):
    # Load the model and explainer if SHAP is enabled
    if display_shap:
        booster_type = model.get_xgb_params().get("booster")
        
        if booster_type == "gblinear":
            sample_size = min(100, len(matches_df))
            # Get all features that were used during model fitting
            features = matches_df[model.feature_names]
            # Scale only numeric features while preserving other columns
            numeric_features = features.select_dtypes(include=['int64', 'float64']).columns
            scaled_features = features.copy()
            scaled_features[numeric_features] = scaler.transform(features[numeric_features])
            masker = shap.maskers.Independent(scaled_features)
            explainer = shap.LinearExplainer(model, masker)
        else:
            explainer = shap.TreeExplainer(model)
        
        if agg_waterfall:
            sample_size = min(100, len(matches_df))
            sample_df = matches_df.sample(n=sample_size, random_state=42)
            features = sample_df[model.feature_names]
            numeric_features = features.select_dtypes(include=['int64', 'float64']).columns
            scaled_features = features.copy()
            scaled_features[numeric_features] = scaler.transform(features[numeric_features])
            shap_values_agg = explainer.shap_values(scaled_features)
            if isinstance(shap_values_agg, list):
                shap_values_agg = shap_values_agg[1]
            plt.figure(figsize=(10, 8))
            shap.summary_plot(
                shap_values_agg,
                scaled_features,
                feature_names=features.columns,
                show=False
            )
            plt.title("Aggregated Feature Importance")
            plt.tight_layout()
            plt.show()

    # Filter and sort based on mode
    if 'label' in matches_df.columns:
        matches_df['prediction'] = (matches_df['proba'] >= 0.5).astype(int)
        
        if mode == 'hard_fn':
            filtered_df = matches_df[(matches_df['label'] == 1) & (matches_df['prediction'] == 0)]
            filtered_df = filtered_df.sort_values('proba', ascending=False)
        elif mode == 'easy_fn':
            filtered_df = matches_df[(matches_df['label'] == 1) & (matches_df['prediction'] == 0)]
            filtered_df = filtered_df.sort_values('proba', ascending=True)
        elif mode == 'hard_fp':
            filtered_df = matches_df[(matches_df['label'] == 0) & (matches_df['prediction'] == 1)]
            filtered_df = filtered_df.sort_values('proba', ascending=True)
        elif mode == 'easy_fp':
            filtered_df = matches_df[(matches_df['label'] == 0) & (matches_df['prediction'] == 1)]
            filtered_df = filtered_df.sort_values('proba', ascending=False)
        elif mode == 'true_matches_head':
            filtered_df = matches_df[matches_df['label'] == 1]
            filtered_df = filtered_df.sort_values('proba', ascending=False)
        elif mode == 'true_matches_tail':
            filtered_df = matches_df[matches_df['label'] == 1]
            filtered_df = filtered_df.sort_values('proba', ascending=True)
        else:  # predicted_matches_head or predicted_matches_tail
            filtered_df = matches_df[matches_df['prediction'] == 1]
            filtered_df = filtered_df.sort_values('proba', ascending=(mode == 'predicted_matches_tail'))
    else:
        # If no labels, only show predicted matches
        filtered_df = matches_df
        filtered_df = filtered_df.sort_values('proba', ascending=(mode == 'predicted_matches_tail'))

    # Group by first SKU to keep related matches together
    for query_idx, (query_sku, query_group) in enumerate(filtered_df.groupby('sku_first')):
        print(f'Query #{query_idx} '+'#' * 40)
        print(f"Query SKU: {query_sku}")
        # if 'url_first' in filtered_df.columns:
        #     print(f'Query URL: {query_group["url_first"].iloc[0]}')

        # if display_name:
        #     print(f"Query Name: {query_group['name_first'].iloc[0]}")

        print('-' * 60)
        
        # Take top_k matches for this query
        for idx, row in query_group.head(top_k).iterrows():
            print(f"Top-{idx % top_k} SKU: {row['sku_second']}")

            if 'url_first' in query_group.columns:
                print(f'\nQuery URL: {query_group["url_first"].iloc[0]}')
            
            if display_name:
                print(f"Query Name: {query_group['name_first'].iloc[0]}")

            if 'url_second' in row:
                print(f"\nTop-{idx % top_k} URL: {row['url_second']}")

            if display_name:
                print(f"Top-{idx % top_k} Name: {row['name_second']}")

            print()
            print(f"Match probability: {row['proba']:.4f}")
            print(f"Predicted label: {1 if row['proba'] >= 0.5 else 0}")
            if 'label' in filtered_df.columns:
                print(f"True label: {row['label']}")
            print()

            if display_stats:
                print("\nQuery and Candidate SKU fields:")
                print("-" * 30)
                fields_dict = {
                    'Query SKU': {
                        'sku': query_sku,
                        'final_price': float(row['final_price_first']),
                        'balance': float(row['balance_first']),
                        'sales': float(row['sales_first']),
                        'rating': float(row['rating_first']),
                        'comments': float(row['comments_first']),
                        'name_sim': float(row['name_sim']),
                        'img_sim': float(row['img_sim']),
                        'desc_sim': float(row['desc_sim']),
                        'opt_sim': float(row['opt_sim'])
                    },
                    'Candidate SKU': {
                        'sku': row['sku_second'],
                        'final_price': float(row['final_price_second']),
                        'balance': float(row['balance_second']),
                        'sales': float(row['sales_second']),
                        'rating': float(row['rating_second']),
                        'comments': float(row['comments_second']),
                        'name_sim': float(row['name_sim']),
                        'img_sim': float(row['img_sim']),
                        'desc_sim': float(row['desc_sim']),
                        'opt_sim': float(row['opt_sim'])
                    }
                }
                pd.set_option('display.float_format', lambda x: '%.2f' % x)
                comparison_df = pd.DataFrame(fields_dict)
                print(comparison_df)
            
            if display_shap:
                features = pd.DataFrame([row[model.feature_names]])
                numeric_features = features.select_dtypes(include=['int64', 'float64']).columns
                scaled_features = features.copy()
                scaled_features[numeric_features] = scaler.transform(features[numeric_features])

                booster_type = model.get_xgb_params().get("booster")
                if booster_type == "gblinear":
                    background_data = matches_df[model.feature_names]
                    scaled_background = background_data.copy()
                    scaled_background[numeric_features] = scaler.transform(background_data[numeric_features])
                    masker = shap.maskers.Independent(scaled_background)
                    explainer = shap.LinearExplainer(model, masker)
                else:
                    explainer = shap.TreeExplainer(model)

                shap_values_row = explainer.shap_values(scaled_features)
                expected_value = explainer.expected_value

                if shap_values:
                    top_features = pd.DataFrame({
                        'Feature': features.columns,
                        'SHAP Value': shap_values_row[0],
                        'Feature Value': features.values[0],
                        'Scaled Value': scaled_features.values[0]
                    }).sort_values(by='SHAP Value', key=abs, ascending=False).head(5)

                    expected_value_float = expected_value.item() if isinstance(expected_value, np.ndarray) else expected_value
                    print(f"\nSHAP explanation for pair:")
                    print(f"Query SKU: {query_sku}")
                    print(f"Candidate SKU: {row['sku_second']}")
                
                if waterfalls:
                    expected_value_float = expected_value.item() if isinstance(expected_value, np.ndarray) else expected_value
                    plt.figure(figsize=(10, 6))
                    title = f"SHAP Values:\nQuery ({query_sku}): {row['name_first']}\nCandidate ({row['sku_second']}): {row['name_second']}"
                    plt.title(title, wrap=True)
                    shap.plots.waterfall(shap.Explanation(
                        values=shap_values_row[0],
                        base_values=expected_value_float,
                        data=scaled_features.values[0],
                        feature_names=features.columns
                    ), max_display=10, show=True)
            
            print()
        print()


In [45]:
def scale_features(df_test, model, scaler):
    """
    Scale the features of the DataFrame using the provided scaler.
    
    Parameters:
    df_test (pd.DataFrame): The DataFrame containing the features to scale.
    scaler: The scaler object used for scaling.
    
    Returns:
    pd.DataFrame: The scaled features.
    """
    # Drop the columns that are not needed for scaling.
    columns_to_drop = [
        'label',

        'sku_first', 'sku_second',

        'name_first', 'description_first',
        'name_second', 'description_second',
        'options_first', 'options_second',

        'image_url_first', 'image_url_second',
        'image_id_first', 'image_id_second',
        'url_first', 'url_second'
        'category_id', 'category_name',
    ]
    
    # Create a DataFrame for scaling by dropping the extra columns.
    X = df_test.drop(columns=columns_to_drop, errors='ignore')
    model.feature_names = X.columns.tolist()
    
    # Ensure the columns match exactly what the scaler was trained on.
    if hasattr(scaler, 'feature_names_in_'):
        expected_features = list(scaler.feature_names_in_)
        X_for_scaler = X[expected_features]
    else:
        X_for_scaler = X
    
    # Scale the features using the loaded scaler.
    X_scaled = scaler.transform(X_for_scaler)
    
    return X_scaled

X_scaled = scale_features(df_custom, model, scaler)

In [46]:
def compute_predictions(model, X_scaled, df):
    # Compute predictions and probabilities (using your pre-trained model)
    predictions = model.predict(X_scaled)
    predicted_probas = model.predict_proba(X_scaled)

    # For binary classification, use the positive class probability;
    # for multiclass, use the highest probability of any class.
    if predicted_probas.shape[1] == 2:
        sort_probas = predicted_probas[:, 1]
    else:
        sort_probas = predicted_probas.max(axis=1)

    # Create a DataFrame with the predictions and probabilities.
    # We align by the original dataframe's index.
    results_df = pd.DataFrame({
        'prediction': predictions,
        'proba': sort_probas
    }, index=df.index)

    # Concatenate the predictions to the original DataFrame.
    df_with_preds = pd.concat([df, results_df], axis=1)

    # Instead of using groupby.apply (with include_groups), sort directly:
    df_grouped_sorted = df_with_preds.sort_values(
        by=['sku_first', 'proba'], ascending=[True, False]
    )

    # print("Data grouped by sku_first and sorted within each group by probability:")
    # print(df_grouped_sorted.columns.tolist())

    matches_df = df_grouped_sorted.copy()
    matches_df = matches_df[matches_df.prediction == 1]

    return df_grouped_sorted, matches_df

results_df, matches_df = compute_predictions(model, X_scaled, df_custom)

In [47]:
# results_df.columns.tolist()
# results_df.groupby('sku_first').prediction.value_counts()

In [48]:
print_samples(
    results_df,

    mode='predicted_matches_head',
    # mode='predicted_matches_tail',
    # mode='hard_fn',
    # mode='easy_fn',
    # mode='hard_fp',
    # mode='easy_fp',
    # mode='true_matches_head',
    # mode='true_matches_tail',

    display_stats=True,
    stats_fmt='text', # or 'table'
    display_name=True,
    # display_shap=True,  # Flag to control SHAP explanation display

    # agg_waterfall=False,  # Whether to show initial aggregated waterfall
    # waterfalls=True,    # Whether to show individual waterfalls
    # shap_values=False    # Whether to print detailed top SHAP values report
)

Query #0 ########################################
Query SKU: 491268805
------------------------------------------------------------
Top-4 SKU: 1573965314

Query URL: https://www.ozon.ru/context/detail/id/491268805/
Query Name: Карта Москвы настенная,100х70 см "Москва современная с линиями метро"

Top-4 URL: https://www.ozon.ru/context/detail/id/1573965314/
Top-4 Name: Настенная карта Москвы "Москва современная с линиями метро"/размер 102х143

Match probability: 0.8108
Predicted label: 1


Query and Candidate SKU fields:
------------------------------
               Query SKU  Candidate SKU
sku         491268805.00  1573965314.00
final_price       416.00        1203.00
balance           233.00          39.00
sales              67.00           0.00
rating              4.80           5.00
comments          606.00           4.00
name_sim            0.91           0.91
img_sim             0.79           0.79
desc_sim            0.83           0.83
opt_sim             0.85           0.85

To

In [166]:
# compare two SKUs
query_sku_first = 64354727  # Replace with the actual SKU
candidate_sku_second = 64129693  # Replace with the actual SKU

# Print numerical fields for query and candidate SKUs
# Get the row for the SKU pair
row = results_df[(results_df['sku_first'] == query_sku_first) & 
                 (results_df['sku_second'] == candidate_sku_second)].iloc[0]

print("\nQuery and Candidate SKU fields:")
print("-" * 30)
# Create a dictionary with the fields to display
fields_dict = {
    'Query SKU': {
        'sku': query_sku_first,
        'final_price': row['final_price_first'],
        'balance': row['balance_first'],
        'sales': row['sales_first'], 
        'rating': row['rating_first'],
        'comments': row['comments_first'],
        'name_sim': row['name_sim'],
        'img_sim': row['img_sim'],
        'desc_sim': row['desc_sim'],
        'opt_sim': row['opt_sim']
    },
    'Candidate SKU': {
        'sku': candidate_sku_second,
        'final_price': row['final_price_second'], 
        'balance': row['balance_second'],
        'sales': row['sales_second'],
        'rating': row['rating_second'], 
        'comments': row['comments_second'],
        'name_sim': row['name_sim'],
        'img_sim': row['img_sim'],
        'desc_sim': row['desc_sim'],
        'opt_sim': row['opt_sim']
    }
}

# Convert to DataFrame and display
comparison_df = pd.DataFrame(fields_dict)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
print(comparison_df)

# Filter the results_df to get the row corresponding to the target pair of SKUs
target_row = results_df[(results_df['sku_first'] == query_sku_first) & 
                       (results_df['sku_second'] == candidate_sku_second)]

# Check if the target pair exists in the DataFrame
if not target_row.empty:
    # Get features for SHAP explanation
    features = target_row[model.feature_names]
    scaled_features = features.copy()
    
    # Get SHAP values
    explainer = shap.TreeExplainer(model)
    shap_values_row = explainer.shap_values(scaled_features)
    expected_value = explainer.expected_value

    # Print top contributing features
    top_features = pd.DataFrame({
        'Feature': features.columns,
        'SHAP Value': shap_values_row[0],
        'Feature Value': features.values[0],
        'Scaled Value': scaled_features.values[0]
    }).sort_values(by='SHAP Value', key=abs, ascending=False).head(5)

    expected_value_float = expected_value.item() if isinstance(expected_value, np.ndarray) else expected_value

    # Plot waterfall
    plt.figure(figsize=(10, 6))
    title = f"SHAP Values:\nQuery ({query_sku_first}): {row['name_first']}\nCandidate ({candidate_sku_second}): {row['name_second']}"
    plt.title(title, wrap=True)
    shap.plots.waterfall(shap.Explanation(
        values=shap_values_row[0],
        base_values=expected_value_float,
        data=scaled_features.values[0],
        feature_names=model.feature_names
    ), max_display=10, show=True)
    
    print(f"\nSHAP explanation for pair:")
    print(f"Query SKU: {query_sku_first}")
    print(f"Candidate SKU: {candidate_sku_second}")
else:
    print(f"No data found for SKU pair: {query_sku_first}, {candidate_sku_second}")



Query and Candidate SKU fields:
------------------------------
              Query SKU  Candidate SKU
sku         64354727.00    64129693.00
final_price      684.00         571.00
balance            0.00          62.00
sales              0.00          20.00
rating             5.00           5.00
comments          97.00         363.00
name_sim           0.87           0.87
img_sim            0.55           0.55
desc_sim           0.74           0.74
opt_sim            0.85           0.85


AttributeError: 'XGBClassifier' object has no attribute 'feature_names'

In [ ]:
row[['options_first', 'options_second']]

options_first           Высота предмета: 107 см. Ширина предмета: 160 см. Тип карты: настенная; географическая; стирающаяся. Вид карты: политическая; Географическая; Настенная. Масштаб: 1:20 000 000. Автор: Атлас Принт. Редакция: атлас принт. Страна производства: Россия. Комплектация: Политическая карта мира 160х107 см - 1 шт. 
options_second    Высота предмета: 112 см. Ширина предмета: 180 см. Тип карты: настенная; Географическая; политическая карта. Вид карты: политическая; Настенная; Географическая. Масштаб: 1:18000000. Автор: Клавдий Птолемей. Редакция: РПГ Карандаш. Страна производства: Россия. Комплектация: Карта мира 186х117 см в тубусе - 1шт. 
Name: 268, dtype: object

# Output files

In [ ]:
def construct_wide_table(df, top_k, skip_no_positives=True):
    """
    Constructs a wide table such that each row corresponds to a unique query SKU and contains:
    
      Query_SKU, Top-1_SKU, Top-2_SKU, ... Top-k_SKU,
      Query_URL, Top-1_URL, Top-2_URL, ... Top-k_URL,
      Top-1_Proba, Top-2_Proba, ... Top-k_Proba
      
    If there are no positive predictions or fewer than top_k positive predictions for a query,
    the remaining columns are filled with None.
    
    Args:
        df: Input DataFrame with predictions
        top_k: Number of top candidates to include
        skip_no_positives: If True, skip query SKUs with no positive predictions
                          If False, include them with all None values for candidates
    """
    wide_rows = []
    # Process each unique query SKU.
    for query_sku, group in df.groupby('sku_first'):
        # Consider only candidates with positive prediction.
        group_positive = group[group['prediction'] == 1]
            
        # Skip if no positives and skip_no_positives is True
        if len(group_positive) == 0 and skip_no_positives:
            continue
            
        # Sort the positive candidates by probability in descending order.
        group_sorted = group_positive.sort_values(by='proba', ascending=False).reset_index(drop=True)
        
        # Get query URL from first row of original group
        query_url = group.iloc[0]['url_first']
        
        # Build the row dictionary with the desired column order.
        row = {}
        row['Query_SKU'] = query_sku
        
        # Add candidate SKU columns.
        for i in range(top_k):
            if i < len(group_sorted):
                row[f'Top-{i+1}_SKU'] = group_sorted.loc[i, 'sku_second']
            else:
                row[f'Top-{i+1}_SKU'] = None
        
        # Add the Query URL.
        row['Query_URL'] = query_url
        
        # Add candidate URL columns.
        for i in range(top_k):
            if i < len(group_sorted):
                row[f'Top-{i+1}_URL'] = group_sorted.loc[i, 'url_second']
            else:
                row[f'Top-{i+1}_URL'] = None
        
        # Add candidate probability columns.
        for i in range(top_k):
            if i < len(group_sorted):
                row[f'Top-{i+1}_proba'] = group_sorted.loc[i, 'proba']
            else:
                row[f'Top-{i+1}_proba'] = None
        
        wide_rows.append(row)
    return pd.DataFrame(wide_rows)


In [ ]:
# --- Set your desired top_k (e.g. 5) and construct the wide table.
TOP_K = 5

matches_wide_df = construct_wide_table(results_df, TOP_K)
# matches_wide_df = matches_wide_df.dropna()
matches_wide_df

,Query_SKU,Top-1_SKU,Top-2_SKU,Top-3_SKU,Top-4_SKU,Top-5_SKU,Query_URL,Top-1_URL,Top-2_URL,Top-3_URL,Top-4_URL,Top-5_URL,Top-1_proba,Top-2_proba,Top-3_proba,Top-4_proba,Top-5_proba
0,491268805,1573965314,484538633,601310061.00,1387125983.00,484577673.00,https://www.ozon.ru/context/detail/id/491268805/,https://www.ozon.ru/context/detail/id/1573965314/,https://www.ozon.ru/context/detail/id/484538633/,https://www.ozon.ru/context/detail/id/601310061/,https://www.ozon.ru/context/detail/id/1387125983/,https://www.ozon.ru/context/detail/id/484577673/,0.89,0.83,0.83,0.81,0.78
1,491270272,1247589234,1005611591,339485530.00,601560330.00,1581230356.00,https://www.ozon.ru/context/detail/id/491270272/,https://www.ozon.ru/context/detail/id/1247589234/,https://www.ozon.ru/context/detail/id/1005611591/,https://www.ozon.ru/context/detail/id/339485530/,https://www.ozon.ru/context/detail/id/601560330/,https://www.ozon.ru/context/detail/id/1581230356/,0.88,0.87,0.86,0.80,0.80
2,491270369,1283826235,178726257,490685952.00,1640637003.00,1857130100.00,https://www.ozon.ru/context/detail/id/491270369/,https://www.ozon.ru/context/detail/id/1283826235/,https://www.ozon.ru/context/detail/id/178726257/,https://www.ozon.ru/context/detail/id/490685952/,https://www.ozon.ru/context/detail/id/1640637003/,https://www.ozon.ru/context/detail/id/1857130100/,0.89,0.89,0.89,0.89,0.88
3,491271284,1887527156,1581328190,1934870477.00,808560063.00,1581230356.00,https://www.ozon.ru/context/detail/id/491271284/,https://www.ozon.ru/context/detail/id/1887527156/,https://www.ozon.ru/context/detail/id/1581328190/,https://www.ozon.ru/context/detail/id/1934870477/,https://www.ozon.ru/context/detail/id/808560063/,https://www.ozon.ru/context/detail/id/1581230356/,0.89,0.88,0.88,0.87,0.87
4,491271320,1436512495,1857130100,1681440240.00,475243659.00,178726257.00,https://www.ozon.ru/context/detail/id/491271320/,https://www.ozon.ru/context/detail/id/1436512495/,https://www.ozon.ru/context/detail/id/1857130100/,https://www.ozon.ru/context/detail/id/1681440240/,https://www.ozon.ru/context/detail/id/475243659/,https://www.ozon.ru/context/detail/id/178726257/,0.91,0.90,0.89,0.88,0.88
5,491271339,1641860393,1710867998,1710805919.00,1573965314.00,1711547819.00,https://www.ozon.ru/context/detail/id/491271339/,https://www.ozon.ru/context/detail/id/1641860393/,https://www.ozon.ru/context/detail/id/1710867998/,https://www.ozon.ru/context/detail/id/1710805919/,https://www.ozon.ru/context/detail/id/1573965314/,https://www.ozon.ru/context/detail/id/1711547819/,0.61,0.60,0.56,0.56,0.55
6,491271768,1710867998,1710805919,1711547819.00,1641860393.00,484538633.00,https://www.ozon.ru/context/detail/id/491271768/,https://www.ozon.ru/context/detail/id/1710867998/,https://www.ozon.ru/context/detail/id/1710805919/,https://www.ozon.ru/context/detail/id/1711547819/,https://www.ozon.ru/context/detail/id/1641860393/,https://www.ozon.ru/context/detail/id/484538633/,0.63,0.63,0.62,0.61,0.60
7,491273438,178733797,178734811,1436528883.00,1436530193.00,1586714576.00,https://www.ozon.ru/context/detail/id/491273438/,https://www.ozon.ru/context/detail/id/178733797/,https://www.ozon.ru/context/detail/id/178734811/,https://www.ozon.ru/context/detail/id/1436528883/,https://www.ozon.ru/context/detail/id/1436530193/,https://www.ozon.ru/context/detail/id/1586714576/,0.80,0.79,0.79,0.79,0.79
8,491273791,1919282160,1436528883,1438364306.00,1919437475.00,625539152.00,https://www.ozon.ru/context/detail/id/491273791/,https://www.ozon.ru/context/detail/id/1919282160/,https://www.ozon.ru/context/detail/id/1436528883/,https://www.ozon.ru/context/detail/id/1438364306/,https://www.ozon.ru/context/detail/id/1919437475/,https://www.ozon.ru/context/detail/id/625539152/,0.88,0.87,0.86,0.86,0.85
9,491279127,1436513974,1438798026,1436684652.00,974286048.00,1581328190.00,https://www.ozon.ru/context/detail/id/491279127/,https://www.ozon.ru/context/detail/id/1436513974/,https://www.ozon.ru/context/detail/id/1438798026/,https://www.ozon.ru/con

In [ ]:
# subset_cols = [
#     'Query_SKU',
#     # 'Top-1_SKU', 'Top-2_SKU', 'Top-3_SKU', 'Top-4_SKU', 'Top-5_SKU',
#     # 'Query_URL',
#     # 'Top-1_URL', 'Top-2_URL', 'Top-3_URL', 'Top-4_URL', 'Top-5_URL',
#     # 'Top-1_proba', 'Top-2_proba', 'Top-3_proba', 'Top-4_proba', 'Top-5_proba'
# ]

# output_file_path = (
#     Path(DATA_FILE).parent /
#     Path('test_results') / 
#     'result-errors-template.csv'
#     # (f'result-errors-template-{MODEL_CKPT_DIR}_' + Path(DATA_FILE).name)
# )
# matches_wide_df.to_clipboard(index=False, header=True, excel=True, columns=subset_cols)
# matches_wide_df.to_csv(output_file_path, index=False, header=True, columns=subset_cols)

In [ ]:
# DATA_PATH = 'data'

# output_file_path = (
#     Path(DATA_FILE).parent /
#     Path('test_results') / 
#     (f'result-{MODEL_CKPT_DIR}_' + Path(DATA_FILE).name)
# )
# output_file_path.parent.mkdir(parents=True, exist_ok=True)
# # output_file_path

# matches_wide_df.to_csv(output_file_path, index=None)